In [ ]:
!pip install Flask numpy tensorflow pillow

In [ ]:
import json
from io import BytesIO

from flask import Flask, request
from tensorflow.keras.models import load_model
from PIL import Image, ImageOps
import numpy as np


with open('env.json') as f:
    env = json.load(f)

app = Flask(__name__)
model = load_model(env['YOUR_HDF5_FILE'])
labels_file = env.get('YOUR_LABELS_FILE', 'labels.txt')
res = 224


@app.route("/", methods=['GET', 'POST'])
def do():
    label = confidence = ''
    
    if request.method == 'POST':
        label = '無圖片'
        if request.files:
            file = request.files.get('image')
            if file:
                image = file.read()
                image = Image.open(BytesIO(image))
                label, c = classify(image)
                confidence = f'{c:.3f}'
    
    page = f'''
    <form method="POST" enctype="multipart/form-data">
        請選擇圖片檔案: <input type="file" name="image">
        <input type="submit" value="預測">
    </form>
    <div>
        標籤: {label}/ 信心: {confidence}
    </div>
    '''
    
    return page

    
def classify(image):
    image = ImageOps.fit(image, (res, res))
    image = (np.expand_dims(image, axis=0).astype(np.float32) / 127.5) - 1
    prediction = model.predict(image)
    i = np.argmax(prediction[0])
    return labels(i), prediction[0][i]


def labels(i):
    with open(labels_file, 'r', encoding='utf-8') as f:
        row = f.readline()
        while row:
            row = row.strip('\n').split()
            if len(row) > 1 and int(row[0]) == i:
                return row[1]
            row = f.readline()
    return '不明'


if __name__ == '__main__':
    app.run()

In [ ]:
!pip install Flask numpy tensorflow pillow

In [ ]:
import json
from io import BytesIO

from flask import Flask, request
from tensorflow.keras.models import load_model
from PIL import Image, ImageOps
import numpy as np


with open('env.json') as f:
    env = json.load(f)

app = Flask(__name__)
model = load_model(env['YOUR_HDF5_FILE'])
labels_file = env.get('YOUR_LABELS_FILE', 'labels.txt')
res = 224


@app.route("/", methods=['GET', 'POST'])
def do():
    label = confidence = ''
    
    if request.method == 'POST':
        label = '無圖片'
        if request.files:
            file = request.files.get('image')
            if file:
                image = file.read()
                image = Image.open(BytesIO(image))
                label, c = classify(image)
                confidence = f'{c:.3f}'
        return {'prediction': label, 'confidence': confidence}
    
    page = '''
    <div id="select">
        請選擇圖片檔案: <input type="file" id="select_img" name="image" accept="image/*, capture=camera" onchange="preview()">
    </div>
    <br>
    <div>
        標籤: <label id="pred"></label>/ 信心: <label id="conf"></label>
    </div>
    <br>
    <img id="preview_img" /><br>
    
    <script language="javascript">    
        function preview() {
            if (!window.FileReader) {
                console.log("no preview functionality supported by your browser!");
                return;
            }
            
            let reader = new FileReader();
            reader.onload = function (event) {
                let img = document.getElementById("preview_img");
                img.src = event.target.result;
                const payload = new FormData();
                payload.append("image", file, file.name);
                fetch('/', {method: "POST", body: payload})
                    .then((response) => response.json())
                    .then((data) => {
                        document.getElementById("pred").innerHTML = data["prediction"]
                        document.getElementById("conf").innerHTML = data["confidence"]
                        console.log(data); 
                    })
                    .catch((error) => {
                        console.log("Error:", error);
                    })
            };

            let file = document.getElementById("select_img").files[0];
            reader.readAsDataURL(file);
        }
    </script>
    '''

    return page


def classify(image):
    image = ImageOps.fit(image, (res, res))
    image = (np.expand_dims(image, axis=0).astype(np.float32) / 127.5) - 1
    prediction = model.predict(image)
    i = np.argmax(prediction[0])
    return labels(i), prediction[0][i]


def labels(i):
    with open(labels_file, 'r', encoding='utf-8') as f:
        row = f.readline()
        while row:
            row = row.strip('\n').split()
            if len(row) > 1 and int(row[0]) == i:
                return row[1]
            row = f.readline()
    return '不明'


if __name__ == '__main__':
    app.run()